# 적응형 RAG (Adaptive RAG)

적응형 RAG는 (1) [질의 분석](https://blog.langchain.dev/query-construction/)과 (2) [능동적/자기 수정 RAG](https://blog.langchain.dev/agentic-rag-with-langgraph/)를 결합한 RAG 전략입니다.

[논문](https://arxiv.org/abs/2403.14403)에서는 다음 3가지 방식 간 라우팅을 위한 질의 분석을 제시합니다:

* 검색 없음 (No Retrieval)
* 단일 단계 RAG (Single-shot RAG)  
* 반복적 RAG (Iterative RAG)

이를 LangGraph를 사용하여 구현해보겠습니다.

우리의 구현에서는 다음 두 방식 간 라우팅을 수행합니다:

* 웹 검색: 최신 사건과 관련된 질의용
* 자기 수정 RAG: 우리의 인덱스와 관련된 질의용

![Screenshot 2024-03-26 at 1.36.03 PM.png](attachment:36fa621a-9d3d-4860-a17c-5d20e6987481.png)

## 📖 구현 개요

본 노트북은 Adaptive-RAG 논문의 핵심 아이디어를 LangGraph로 구현하여 다음을 보여줍니다:
- 질의 복잡도에 따른 적응형 처리
- 품질 보장을 위한 다단계 검증 시스템
- 자동 질의 개선 및 재시도 메커니즘

## 설정 (Setup)

먼저 필요한 패키지들을 설치하고 API 키를 설정합니다

In [ ]:
# 패키지 설치 (출력 숨김)
%%capture --no-stderr
%pip install -U langchain_community tiktoken langchain-openai langchain-cohere langchainhub chromadb langchain langgraph  tavily-python

In [ ]:
# API 키 설정
import getpass
import os

def _set_env(var: str):
    """환경변수 설정 함수 - 이미 설정되지 않은 경우에만 입력 요청"""
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

# 필수 API 키들 설정
_set_env("OPENAI_API_KEY")     # OpenAI GPT 모델 사용을 위한 키
# _set_env("COHERE_API_KEY")   # Cohere 임베딩 사용시 필요 (선택사항)
_set_env("TAVILY_API_KEY")     # 웹 검색 기능을 위한 Tavily API 키

<div class="admonition tip">
    <p class="admonition-title">LangGraph 개발을 위한 <a href="https://smith.langchain.com">LangSmith</a> 설정</p>
    <p style="padding-top: 5px;">
        LangSmith에 가입하여 LangGraph 프로젝트의 문제를 신속하게 발견하고 성능을 개선하세요. LangSmith를 사용하면 트레이스 데이터로 LangGraph로 구축한 LLM 앱을 디버그, 테스트, 모니터링할 수 있습니다 — 시작 방법에 대한 자세한 내용은 <a href="https://docs.smith.langchain.com">여기</a>를 참조하세요. 
    </p>
</div>

## 인덱스 생성 (Create Index)

OpenAI Embeddings와 Chroma 벡터 데이터베이스를 사용하여 벡터 데이터베이스를 설정합니다.  
에이전트, 프롬프트 엔지니어링, 대형 언어 모델(LLM)과 관련된 블로그 게시물의 URL을 입력합니다.  
검색 증강 생성(RAG)에 사용할 벡터 인덱스를 생성합니다.

In [ ]:
### 인덱스 구축

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

### from langchain_cohere import CohereEmbeddings  # Cohere 임베딩 사용시

# 임베딩 모델 설정 (OpenAI 임베딩 사용)
embd = OpenAIEmbeddings()

# 인덱싱할 문서들의 URL 목록
# Lilian Weng의 유명한 AI/ML 블로그 포스트들
urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",          # AI 에이전트에 관한 포스트
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/", # 프롬프트 엔지니어링
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",     # LLM 공격 기법
]

# 웹 페이지 로딩
print("웹 페이지 로딩 중...")
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]  # 리스트 평탄화
print(f"총 {len(docs_list)}개 문서 로딩 완료")

# 텍스트 분할 (tiktoken 기반 - OpenAI 토크나이저 사용)
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500,    # 각 청크의 최대 토큰 수
    chunk_overlap=0    # 청크 간 겹치는 토큰 수 (0으로 설정)
)
doc_splits = text_splitter.split_documents(docs_list)
print(f"총 {len(doc_splits)}개 청크로 분할 완료")

# Chroma 벡터스토어에 문서 추가 및 검색기 생성
print("벡터 인덱스 생성 중...")
vectorstore = Chroma.from_documents(
    documents=doc_splits,           # 분할된 문서들
    collection_name="rag-chroma",  # 컬렉션 이름
    embedding=embd,                 # 임베딩 모델
)
retriever = vectorstore.as_retriever()  # 검색기 객체 생성
print("인덱스 생성 완료!")

## LLM 설정

<div class="admonition note">
    <p class="admonition-title">LangChain에서 Pydantic 사용</p>
    <p>
        이 노트북은 Pydantic v2 <code>BaseModel</code>을 사용하므로 <code>langchain-core >= 0.3</code>이 필요합니다. <code>langchain-core < 0.3</code> 사용 시 Pydantic v1과 v2 <code>BaseModel</code>의 혼재로 인한 오류가 발생합니다.
    </p>
</div>

### 질의 분석을 위한 라우터 (Router for Query Analysis)

라우팅부터 시작해보겠습니다. 먼저 LLM에 질의 분석을 할당합니다.

RouteQuery 데이터 모델을 생성하고 LLM에 구조화된 형식으로 지정합니다. 라우팅 결정은 프롬프트에 포함되어야 합니다. 문서의 어떤 부분을 주제에 따라 RAG로 연결할지 명확히 정의해야 합니다.

LLM이 RAG 문서를 다시 요약하도록 자동화할 수 있지만, 대용량 문서를 다룰 때는 자동화가 비용이 많이 들 수 있으므로 수동으로 관리하는 것이 더 비용 효율적입니다.

In [ ]:
# 테스트 실행
print("=== 라우터 테스트 ===")
print("질문 1: 'Who will the Bears draft first in the NFL draft?'")
result1 = question_router.invoke({"question": "Who will the Bears draft first in the NFL draft?"})
print(f"라우팅 결과: {result1.datasource}")
print("\n질문 2: 'What are the types of agent memory?'")
result2 = question_router.invoke({"question": "What are the types of agent memory?"})
print(f"라우팅 결과: {result2.datasource}")

datasource='web_search'
datasource='vectorstore'

### 검색 평가기 (Retrieval Grader)

검색을 수행한 후 결과를 평가합니다. 질의에 따라 RAG를 사용하기로 처음에 결정했지만, 검색된 문서들이 만족스럽지 않을 수 있습니다. 검색된 문서들이 질의에 충분히 관련이 있는지 평가합니다.

이를 위해 LLM에 의존하여 관련성을 평가하고, 이진 'yes' 또는 'no' 결정을 제공합니다.

In [ ]:
print(f"\n평가 결과: {grade_result.binary_score}")

binary_score='yes'

print(f"\n생성된 답변:\n{generation}")

Agent memory in LLM-powered autonomous systems consists of short-term and long-term memory. Short-term memory utilizes in-context learning for immediate tasks, while long-term memory allows agents to retain and recall information over extended periods, often using external storage for efficient retrieval. This memory structure supports the agent's ability to reflect on past actions and improve future performance.

In [ ]:
### 답변 생성 체인 구현

from langchain import hub
from langchain_core.output_parsers import StrOutputParser

# LangChain Hub에서 검증된 RAG 프롬프트 가져오기
# 이 프롬프트는 컨텍스트와 질문을 받아 답변을 생성하도록 설계됨
prompt = hub.pull("rlm/rag-prompt")

# LLM 설정 (답변 생성용)
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# 문서 목록을 하나의 텍스트로 결합하는 함수
def format_docs(docs):
    """검색된 문서들을 하나의 컨텍스트 문자열로 결합
    
    Args:
        docs: 검색된 Document 객체들의 리스트
        
    Returns:
        str: 각 문서 내용을 두 줄 간격으로 결합한 문자열
    """
    return "\n\n".join(doc.page_content for doc in docs)

# RAG 체인 구성: 프롬프트 → LLM → 문자열 파서
rag_chain = prompt | llm | StrOutputParser()

# 테스트 실행
print("=== RAG 체인 테스트 ===")
docs_txt = format_docs(docs)  # 이전에 검색한 문서들 사용
print(f"컨텍스트 길이: {len(docs_txt)} 문자")
print(f"질문: {question}")

# 답변 생성
generation = rag_chain.invoke({"context": docs_txt, "question": question})
print(f"\n생성된 답변:\n{generation}")

### 환각 평가기 (Hallucination Grader)

LLM이 검색된 사실과 비교하여 환각을 생성했는지 확인합니다.  
LLM의 평가를 이진 'yes' 또는 'no' 형식으로 제공합니다.

In [ ]:
    print("❌ 답변에 환각이나 근거 없는 내용이 포함되어 있습니다.")

GradeHallucinations(binary_score='yes')

### 답변 평가기 (Answer Grader)

마지막으로 생성된 답변이 원래 질문에 적절히 답변하는지 평가합니다.

In [ ]:
    print("❌ 답변이 질문을 충분히 해결하지 못합니다.")

GradeAnswer(binary_score='yes')

### 질의 재작성 (Question Rewriting)

사용자의 원래 질문이 RAG에서 직접 사용되었습니다.  
하지만 사용자의 질문이 RAG에 적합한 형태가 아닐 수 있습니다.  
검색을 개선하기 위해 벡터 유사성 검색에 더 적합하도록 질문을 다시 표현합니다.

In [ ]:
print(f"개선된 질문: {rewritten_question}")

'What are the key concepts and techniques related to agent memory in artificial intelligence?'

## 웹 검색 도구 (Web Search Tool)

웹에서 정보를 얻기 위해 Tavily Search 도구를 사용합니다.

In [ ]:
### 웹 검색 도구 설정

from langchain_community.tools.tavily_search import TavilySearchResults

# Tavily 웹 검색 도구 초기화
# k=3: 상위 3개 검색 결과만 반환
web_search_tool = TavilySearchResults(k=3)

print("✅ 웹 검색 도구 설정 완료")
print("- 검색 엔진: Tavily")
print("- 최대 결과 수: 3개")
print("- 사용 목적: 최신 정보 및 실시간 데이터 검색")

## 그래프 구성 (Construct the Graph)

흐름을 그래프로 캡처합니다.

### 그래프 상태 정의 (Define Graph State)

In [ ]:
### 그래프 상태 정의

from typing import List
from typing_extensions import TypedDict

class GraphState(TypedDict):
    """
    우리 그래프의 상태를 나타냅니다.

    Attributes:
        question: 사용자 질문
        generation: LLM 생성 답변
        documents: 검색된 문서 리스트
    """

    question: str          # 현재 처리 중인 질문
    generation: str        # 생성된 답변
    documents: List[str]   # 검색된 관련 문서들

print("✅ 그래프 상태 클래스 정의 완료")
print("상태 구성 요소:")
print("- question: 사용자의 질문")
print("- generation: AI가 생성한 답변")
print("- documents: 검색된 참조 문서들")

### 그래프 플로우 정의 (Define Graph Flow)

각 노드와 엣지의 동작을 정의합니다.

In [ ]:
### 그래프 노드 함수들 정의

from pprint import pprint
from langchain.schema import Document

# === 핵심 처리 노드들 ===

def retrieve(state):
    """
    벡터스토어에서 문서 검색

    Args:
        state (dict): 현재 그래프 상태

    Returns:
        state (dict): documents 키에 검색된 문서들이 추가된 새로운 상태
    """
    print("---문서 검색 수행---")
    question = state["question"]
    
    # 벡터스토어에서 관련 문서 검색
    documents = retriever.invoke(question)
    print(f"검색된 문서 수: {len(documents)}개")
    
    return {"documents": documents, "question": question}


def generate(state):
    """
    검색된 문서들을 바탕으로 답변 생성

    Args:
        state (dict): 현재 그래프 상태

    Returns:
        state (dict): generation 키에 LLM 생성 답변이 추가된 새로운 상태
    """
    print("---답변 생성 수행---")
    question = state["question"]
    documents = state["documents"]

    # RAG 방식으로 답변 생성
    docs_txt = format_docs(documents)
    generation = rag_chain.invoke({"context": docs_txt, "question": question})
    print(f"생성된 답변 길이: {len(generation)}자")
    
    return {"documents": documents, "question": question, "generation": generation}


def grade_documents(state):
    """
    검색된 문서들이 질문과 관련성이 있는지 판단합니다.

    Args:
        state (dict): 현재 그래프 상태

    Returns:
        state (dict): 관련성이 있는 문서들만 필터링하여 documents 키 업데이트
    """
    print("---문서 관련성 검사---")
    question = state["question"]
    documents = state["documents"]

    # 각 문서별로 관련성 점수 계산
    filtered_docs = []
    for i, d in enumerate(documents):
        score = retrieval_grader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score.binary_score
        if grade == "yes":
            print(f"---평가: 문서 {i+1} 관련성 있음---")
            filtered_docs.append(d)
        else:
            print(f"---평가: 문서 {i+1} 관련성 없음---")
            continue
    
    print(f"필터링 결과: {len(documents)}개 → {len(filtered_docs)}개")
    return {"documents": filtered_docs, "question": question}


def transform_query(state):
    """
    더 나은 검색을 위해 질문을 변환합니다.

    Args:
        state (dict): 현재 그래프 상태

    Returns:
        state (dict): 재작성된 질문으로 question 키 업데이트
    """
    print("---질의 변환 수행---")
    question = state["question"]
    documents = state["documents"]

    # 질문 재작성
    better_question = question_rewriter.invoke({"question": question})
    print(f"원래 질문: {question}")
    print(f"개선된 질문: {better_question}")
    
    return {"documents": documents, "question": better_question}


def web_search(state):
    """
    재작성된 질문을 바탕으로 웹 검색을 수행합니다.

    Args:
        state (dict): 현재 그래프 상태

    Returns:
        state (dict): 웹 검색 결과로 documents 키 업데이트
    """
    print("---웹 검색 수행---")
    question = state["question"]

    # 웹 검색 실행
    docs = web_search_tool.invoke({"query": question})
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)
    print(f"웹 검색 결과 길이: {len(web_results.page_content)}자")

    return {"documents": web_results, "question": question}


# === 조건부 분기 함수들 ===

def route_question(state):
    """
    질문을 웹 검색 또는 RAG로 라우팅합니다.

    Args:
        state (dict): 현재 그래프 상태

    Returns:
        str: 호출할 다음 노드 이름
    """
    print("---질의 라우팅---")
    question = state["question"]
    source = question_router.invoke({"question": question})
    
    if source.datasource == "web_search":
        print("---질의를 웹 검색으로 라우팅---")
        return "web_search"
    elif source.datasource == "vectorstore":
        print("---질의를 RAG로 라우팅---")
        return "vectorstore"


def decide_to_generate(state):
    """
    답변 생성을 할지, 아니면 질문을 재생성할지 결정합니다.

    Args:
        state (dict): 현재 그래프 상태

    Returns:
        str: 호출할 다음 노드에 대한 이진 결정
    """
    print("---평가된 문서들 검토---")
    filtered_documents = state["documents"]

    if not filtered_documents:
        # 모든 문서가 관련성 검사에서 필터링됨
        # 새로운 질의를 재생성할 것
        print("---결정: 모든 문서가 질문과 관련 없음, 질의 변환 수행---")
        return "transform_query"
    else:
        # 관련 문서가 있으므로 답변 생성
        print("---결정: 관련 문서 존재, 답변 생성---")
        return "generate"


def grade_generation_v_documents_and_question(state):
    """
    생성된 답변이 문서에 근거하고 있고 질문에 답변하는지 결정합니다.

    Args:
        state (dict): 현재 그래프 상태

    Returns:
        str: 호출할 다음 노드에 대한 결정
    """
    print("---환각 검사---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]

    # 환각 검사
    score = hallucination_grader.invoke(
        {"documents": documents, "generation": generation}
    )
    grade = score.binary_score

    if grade == "yes":
        print("---결정: 생성 결과가 문서에 근거함---")
        # 질문-답변 적합성 검사
        print("---질문 대비 생성 결과 평가---")
        score = answer_grader.invoke({"question": question, "generation": generation})
        grade = score.binary_score
        if grade == "yes":
            print("---결정: 생성 결과가 질문에 적절히 답변함---")
            return "useful"
        else:
            print("---결정: 생성 결과가 질문에 답변하지 못함---")
            return "not useful"
    else:
        print("---결정: 생성 결과가 문서에 근거하지 않음, 재시도---")
        return "not supported"

print("✅ 모든 그래프 노드 함수 정의 완료")

### 그래프 컴파일 (Compile Graph)

정의된 노드들과 엣지들을 하나의 실행 가능한 워크플로로 조합합니다.

In [ ]:
### 그래프 워크플로 구성 및 컴파일

from langgraph.graph import END, StateGraph, START

# StateGraph 인스턴스 생성
workflow = StateGraph(GraphState)

# === 노드 추가 ===
# 각 노드는 특정 기능을 수행하는 함수와 연결됩니다
workflow.add_node("web_search", web_search)           # 웹 검색 노드
workflow.add_node("retrieve", retrieve)               # 벡터스토어 검색 노드
workflow.add_node("grade_documents", grade_documents) # 문서 관련성 평가 노드
workflow.add_node("generate", generate)               # 답변 생성 노드
workflow.add_node("transform_query", transform_query) # 질의 변환 노드

# === 엣지 구성 ===
# 조건부 엣지: 시작점에서 라우팅 결정
workflow.add_conditional_edges(
    START,           # 시작점
    route_question,  # 라우팅 함수
    {
        "web_search": "web_search",    # 웹 검색 경로
        "vectorstore": "retrieve",     # 벡터스토어 검색 경로
    },
)

# 고정 엣지: 웹 검색 → 답변 생성
workflow.add_edge("web_search", "generate")

# 고정 엣지: 문서 검색 → 문서 평가
workflow.add_edge("retrieve", "grade_documents")

# 조건부 엣지: 문서 평가 결과에 따른 분기
workflow.add_conditional_edges(
    "grade_documents",    # 문서 평가 노드
    decide_to_generate,   # 생성 여부 결정 함수
    {
        "transform_query": "transform_query",  # 질의 변환 경로
        "generate": "generate",                # 답변 생성 경로
    },
)

# 고정 엣지: 질의 변환 → 재검색
workflow.add_edge("transform_query", "retrieve")

# 조건부 엣지: 답변 생성 후 품질 평가
workflow.add_conditional_edges(
    "generate",                                    # 답변 생성 노드
    grade_generation_v_documents_and_question,    # 품질 평가 함수
    {
        "not supported": "generate",        # 재생성 (환각 발견시)
        "useful": END,                      # 종료 (성공)
        "not useful": "transform_query",   # 질의 변환 후 재시도
    },
)

# 워크플로 컴파일 - 실행 가능한 앱으로 변환
app = workflow.compile()

print("✅ 적응형 RAG 워크플로 컴파일 완료")
print("\n🔄 워크플로 구조:")
print("1. START → 질의 라우팅 (웹검색 vs 벡터스토어)")
print("2. 벡터스토어 경로: 검색 → 문서평가 → 생성 여부 결정")
print("3. 웹검색 경로: 웹검색 → 답변생성")
print("4. 품질 검증: 환각검사 → 답변적합성검사 → END")
print("5. 자동 재시도: 품질 미달시 질의변환 → 재검색")

## 그래프 사용 (Use Graph)

구성된 적응형 RAG 시스템을 실제로 테스트해봅니다.

In [ ]:
# 최종 생성 결과 출력
print("📝 최종 답변:")
pprint(value["generation"])
print("\n" + "="*50 + "\n")

---ROUTE QUESTION---
---ROUTE QUESTION TO WEB SEARCH---
---WEB SEARCH---
"Node 'web_search':"
'\n---\n'
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
"Node 'generate':"
'\n---\n'
('The Chicago Bears are expected to draft quarterback Caleb Williams first '
 'overall in the 2024 NFL Draft. They also have a second first-round pick, '
 'where they selected wide receiver Rome Odunze.')

In [ ]:
print("- 자동 라우팅 및 자기 수정 메커니즘 확인됨")

---ROUTE QUESTION---
---ROUTE QUESTION TO RAG---
---RETRIEVE---
"Node 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
"Node 'grade_documents':"
'\n---\n'
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
"Node 'generate':"
'\n---\n'
('The types of agent memory include short-term memory, long-term memory, and '
 'sensory memory. Short-term memory is utilized for in-context learning, while '
 'long-term memory allows for the retention and recall of information over '
 'extended periods. Sensory memory involves learning embedding representations '
 'for various raw inputs, such as text and images.')